In [23]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

# 将上一级目录添加到模块搜索路径
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of models failed: Traceback (most recent call last):
  File "/Users/jiwenyu/Dev/jwgen/jwenv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/jiwenyu/Dev/jwgen/jwenv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/jiwenyu/Dev/jwgen/jwenv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/jiwenyu/Dev/jwgen/jwenv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 335, in update_class
    if (old_obj == new_obj) is True:
        ^^^^^^^^^^^^^^^^^^
  File "/Users/jiwenyu/Dev/jwgen/jwenv/lib/python3.11/site-packages/sqlalchemy/sql/operators.py", line 582, in __eq__
    return self.operate(eq, other)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jiwenyu/Dev/jwgen/jwenv/lib/python3.11/site-packages/sqlalc

In [24]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import utility

In [25]:
connections.connect(alias="default", host="localhost", port="19530")

collection_name = "paper_collection"

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="paper_id", dtype=DataType.INT64),  # 论文 ID
    FieldSchema(name="chunk_id", dtype=DataType.INT64),  # 论文中的 chunk 序号
    FieldSchema(name="chunk_text", dtype=DataType.VARCHAR, max_length=5000),  # chunk 原始文本
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=768),  # chunk 向量
    FieldSchema(name="chunk_type", dtype=DataType.VARCHAR, max_length=50),  # chunk 类型 (如 abstract, conclusion)
]
schema = CollectionSchema(fields, description="Collection for paper embeddings")

vdb_collection = None
if not utility.has_collection(collection_name):
    vdb_collection = Collection(name=collection_name, schema=schema)
    print(f"Collection {collection_name} created.")
else:
    vdb_collection = Collection(name=collection_name)
    print(f"Collection {collection_name} already exists.")


Collection paper_collection already exists.


In [26]:
from repositories import EmbeddingRepository
embedding_repository = EmbeddingRepository(vdb_collection)

In [32]:
# 获取集合中的总实体数
total_entities = vdb_collection.num_entities
print(f"Total records in collection '{collection_name}': {total_entities}")

Total records in collection 'paper_collection': 46


In [ ]:
# 删除 collection 中的所有实体数据
vdb_collection.delete()

In [28]:
# 处理元数据
import pandas as pd

# 读取 CSV 文件
csv_file = "../data/papers_metadata.csv" 
metadata_df = pd.read_csv(csv_file)

# 打印数据表格
print(metadata_df.columns.to_list())
# CSV标题：paper_title,author_ids,author_names,venue,research_area,keywords,tldr,abstract,url,pdf_url,attachment_url,pdf_path

# use a for loop to print first 2 row all data columns
for index, row in metadata_df[:2].iterrows():
     print(row["paper_title"])
     print(row["abstract"])
     print(row["author_names"].split(","))
     print(row["author_ids"].split(","))
     print(row["pdf_path"])
     print(row["tldr"])
     print(row["venue"])
     print(row["research_area"])
     print(row["keywords"].split(","))
     print(row["url"])
     print("-" * 80)


['paper_title', 'author_ids', 'author_names', 'venue', 'research_area', 'keywords', 'tldr', 'abstract', 'url', 'pdf_url', 'attachment_url', 'pdf_path']
Stress-Testing Capability Elicitation With Password-Locked Models
To determine the safety of large language models (LLMs), AI developers must be able to assess their dangerous capabilities. But simple prompting strategies often fail to elicit an LLM’s full capabilities. One way to elicit capabilities more robustly is to fine-tune the LLM to complete the task. In this paper, we investigate the conditions under which fine-tuning-based elicitation suffices to elicit capabilities. To do this, we introduce password-locked models, LLMs fine-tuned such that some of their capabilities are deliberately hidden. Specifically, these LLMs are trained to exhibit these capabilities only when a password is present in the prompt, and to imitate a much weaker LLM otherwise. Password-locked models enable a novel method of evaluating capabilities elicitati

In [29]:
from llm.ollama_model import OllamaModel
from paper_agent import PDFAnalyzer

# create ollama model
llm = OllamaModel()

for index, row in metadata_df[:1].iterrows():
    pdf_path = "../data/pdfs/neurips/2024/Stress_Testing_Capability_Elicitation_With_Password_Locked_Models.pdf"
    
    # Create a PDFAnalyzer object
    # CSV标题：paper_title,author_ids,author_names,venue,research_area,keywords,tldr,abstract,url,pdf_url,attachment_url,pdf_path

    pdf_analyzer = PDFAnalyzer(
        title= row["paper_title"],
        abstract= row["abstract"],
        author_names= row["author_names"].split(","),
        author_ids= row["author_ids"].split(","),
        pdf_path=pdf_path,
        tldr=row["tldr"],
        keywords=row["keywords"].split(","),
        llm=llm
    )

In [33]:
text = pdf_analyzer.extract_text(pdf_path=pdf_analyzer.pdf_path)
text_lines =pdf_analyzer.extract_text_lines(text)
title_indices = pdf_analyzer.get_section_title_indices(text_lines)
section_lines_dict = pdf_analyzer.split_text_lines_by_section_title(text_lines, title_indices)

embedding_list = pdf_analyzer.embed_main_text(llm, section_lines_dict)

INFO:paper_agent:Converted PDF to text: Stress-Testing Capability Elicitation With
Password-Locked Models

Ryan Greenblatt∗
Redwood Research

INFO:paper_agent:Detected title: limitations at line 1120
INFO:paper_agent:Split text into 42 chunks
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "

In [34]:
embedding_repository.insert_chunks_batch(2,embedding_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
['abstract', 'introduction', 'introduction', 'introduction', 'introduction', 'introduction', 'introduction', 'related_work', 'related_work', 'experiment', 'experiment', 'experiment', 'experiment', 'experiment', 'experiment', 'experiment', 'experiment', 'experiment', 'experiment', 'experiment', 'methodology', 'methodology', 'methodology', 'methodology', 'limitations', 'limitations', 'conclusion', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0', 'limitations_0']


INFO:repositories.embedding_repository:Successfully inserted 42 chunks for paper 2


True